In [6]:
import tkinter
from rich import print

In [7]:
# TO DO
# add ERROR message
# clean up 
# tkinter
# round loads up to nearest int [DONE]


''' Limitations:
1. No live load reduction
2. Will only list governing load combo (so if tension governs, you will only get that value and vice versa). 
Must check uplift manually if using lateral columns.
3. No skip live loadings
4. Concrete columns from RAM only.
'''

' Limitations:\n1. No live load reduction\n2. Will only list governing load combo (so if tension governs, you will only get that value and vice versa). \nMust check uplift manually if using lateral columns.\n3. No skip live loadings\n4. Concrete columns from RAM only.\n'

In [8]:
import pandas as pd
import numpy as np
import csv
import openpyxl
import math

In [10]:
# db_path = '17605_20240318_cma_Lateral Columns_Long East Wall_24x24 Int_jrg testing_TYP COL SIZE_EXCEL_story_col_data.xlsx'

# column_db_story_data = pd.read_excel(db_path, sheet_name='StoryData')
# columns = ['Story_UID', 'Level', 'Story_Label', 'FloorHeight', 'CummulativeHeight']
# column_db_story_data = column_db_story_data[columns]

In [11]:
column_db_story_data

NameError: name 'column_db_story_data' is not defined

In [13]:
column_db_column_data = pd.read_excel(db_path, sheet_name='ColumnData')
columns = ['Story_Label', 'x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'NodeI', 'NodeJ', 'Fy_FPC_E', 'Size', 'Grid_Label']
column_db_column_data = column_db_column_data[columns]

In [14]:
column_db_column_data.head()

,Story_Label,x1,y1,z1,x2,y2,z2,NodeI,NodeJ,Fy_FPC_E,Size,Grid_Label
0,1ST FLOOR,-0.580000,-0.580000,1.0,-0.580000,-0.580000,0.0,1802,1861,10,16 x 24,-0.58ft - -0.58ft
1,1ST FLOOR,-0.503917,18.841042,1.0,-0.503917,18.841042,0.0,1803,1862,10,16 x 24,-0.50ft - 18.84ft
2,1ST FLOOR,1.016250,90.908169,1.0,1.016250,90.908169,0.0,1807,1866,10,20 x 24,1.02ft - 90.91ft
3,1ST FLOOR,1.211667,100.171586,1.0,1.211667,100.171586,0.0,1808,1867,10,20 x 24,1.21ft - 100.17ft
4,1ST FLOOR,15.340542,-1.025625,1.0,15.340542,-1.025625,0.0,1813,1872,10,16 x 24,15.34ft - -1.03ft


In [15]:
column_db_column_data.sort_values(['Grid_Label', 'Story_Label']).head(20)

,Story_Label,x1,y1,z1,x2,y2,z2,NodeI,NodeJ,Fy_FPC_E,Size,Grid_Label
1,1ST FLOOR,-0.503917,18.841042,1.000000,-0.503917,18.841042,0.000000,1803,1862,10,16 x 24,-0.50ft - 18.84ft
42,1ST MEZZANINE,-0.503917,18.841042,10.750000,-0.503917,18.841042,1.000000,1744,1803,10,16 x 24,-0.50ft - 18.84ft
1187,BULKHEAD,-0.503917,18.841042,399.500012,-0.503917,18.841042,378.000011,15,57,6,16 x 24,-0.50ft - 18.84ft
362,LEVEL 10,-0.503917,18.841042,111.000003,-0.503917,18.841042,100.000003,1264,1314,10,16 x 24,-0.50ft - 18.84ft
395,LEVEL 11,-0.503917,18.841042,122.000004,-0.503917,18.841042,111.000003,1214,1264,8,16 x 24,-0.50ft - 18.84ft
428,LEVEL 12,-0.503917,18.841042,133.000004,-0.503917,18.841042,122.000004,1164,1214,8,16 x 24,-0.50ft - 18.84ft
461,LEVEL 13,-0.503917,18.841042,144.000004,-0.503917,18.841042,133.000004,1114,1164,8,16 x 24,-0.50ft - 18.84ft
494,LEVEL 14,-0.503917,18.841042,155.000005,-0.503917,18.841042,144.000004,1064,1114,8,16 x 24,-0.50ft - 18.84ft
527,LEVEL 15,-0.503917,18.841042,166.000005,-0.503917,18.841042,155.000005,1014,1064,8,16 x 24,-0.50ft - 18.84ft
560,LEVEL 16,-0.503917,18.841042,177.000005,-0.503917,18.841042,166.000005,964,1014,8,16 x 24,-0.50ft - 18.84ft


In [9]:
with open(r'./test_data/17605_20240328_dk_col_design.csv', 'r') as cdf:
    reader = csv.reader(cdf)
    
    info = {}
    raw_data =[]
    level = []
    grid_loc = []
    size = []
    rebar = []
    pu = []
    for i, row in enumerate(reader):
#         print(row)
        # KEYS
        if 'Level.' in row:
            raw_data.append(row[1])
            level.append(row[1])
        if 'Column Number:.' in row:
            raw_data.append(row[3])
            grid_loc.append(row[3])
            
        # VALUES  
        if 'Size:.' in row:
            raw_data.append(row[1].split(' ')[1])
            size.append(row[1].split(' ')[1])
        if 'Longitudinal:.' in row:
            raw_data.append(row[1].strip().split(' ')[0])
            rebar.append(row[1].strip().split(' ')[0])
        if 'Axial' in row:
            raw_data.append(math.ceil(float(row[3])))
            pu.append(math.ceil(float(row[3])))
            
    cdf.close()
    
    print(len(level), len(grid_loc), len(size), len(rebar), len(pu))
    

1211 1211 1211 1211 1211

In [10]:
print(raw_data)

[
    'BULKHEAD ROOF',
    '(45.45ft-76.54ft)',
    '16',
    '12-#6',
    68,
    'BULKHEAD ROOF',
    '(45.45ft-55.06ft)',
    '16',
    '12-#6',
    97,
    'BULKHEAD',
    '(24.41ft-90.91ft)',
    '16',
    '12-#6',
    96,
    'BULKHEAD',
    '(110.64ft-90.91ft)',
    '16',
    '12-#6',
    51,
    'BULKHEAD',
    '(110.83ft-76.45ft)',
    '16',
    '12-#6',
    34,
    'BULKHEAD',
    '(111.09ft-46.16ft)',
    '16',
    '12-#6',
    67,
    'BULKHEAD',
    '(110.90ft-32.83ft)',
    '16',
    '12-#6',
    45,
    'BULKHEAD',
    '(90.83ft-90.91ft)',
    '16',
    '12-#6',
    76,
    'BULKHEAD',
    '(90.64ft-76.40ft)',
    '16',
    '12-#6',
    60,
    'BULKHEAD',
    '(91.44ft-46.16ft)',
    '16',
    '12-#6',
    78,
    'BULKHEAD',
    '(91.44ft-32.83ft)',
    '16',
    '12-#6',
    59,
    'BULKHEAD',
    '(70.24ft-90.91ft)',
    '16',
    '12-#6',
    67,
    'BULKHEAD',
    '(70.26ft-76.05ft)',
    '16',
    '12-#6',
    84,
    'BULKHEAD',
    '(71.24ft-57.01ft)',
    '16',
    '12-#6',
    132,
    'BULKHEAD',
    '(73.18ft-32.83ft)',
    '16',
    '12-#6',
    129,
    'BULKHEAD',
    '(45.47ft-90.94ft)',
    '16',
    '12-#6',
    67,
    'BULKHEAD',
    '(45.45ft-76.54ft)',
    '16',
    '12-#6',
    112,
    'BULKHEAD',
    '(45.45ft-55.06ft)',
    '16',
    '12-#6',
    192,
    'BULKHEAD',
    '(45.45ft-32.83ft)',
    '16',
    '12-#6',
    140,
    'BULKHEAD',
    '(30.85ft-18.85ft)',
    '16',
    '12-#6',
    80,
    'BULKHEAD',
    '(30.85ft--1.14ft)',
    '16',
    '12-#6',
    53,
    'BULKHEAD',
    '(15.81ft-18.85ft)',
    '16',
    '12-#6',
    90,
    'BULKHEAD',
    '(15.34ft--1.03ft)',
    '16',
    '12-#6',
    31,
    'BULKHEAD',
    '(-0.50ft-18.84ft)',
    '16',
    '12-#6',
    65,
    'BULKHEAD',
    '(-0.58ft--0.58ft)',
    '16',
    '12-#6',
    60,
    'ROOF LEVEL',
    '(123.03ft-104.83ft)',
    '16',
    '12-#6',
    26,
    'ROOF LEVEL',
    '(121.81ft-46.87ft)',
    '16',
    '12-#6',
    21,
    'ROOF LEVEL',
    '(121.51ft-32.71ft)',
    '16',
    '12-#6',
    26,
    'ROOF LEVEL',
    '(110.46ft-105.17ft)',
    '16',
    '12-#6',
    45,
    'ROOF LEVEL',
    '(110.64ft-90.91ft)',
    '16',
    '12-#6',
    126,
    'ROOF LEVEL',
    '(110.83ft-76.45ft)',
    '16',
    '12-#6',
    77,
    'ROOF LEVEL',
    '(111.09ft-46.16ft)',
    '16',
    '12-#6',
    137,
    'ROOF LEVEL',
    '(110.90ft-32.83ft)',
    '16',
    '12-#6',
    97,
    'ROOF LEVEL',
    '(91.01ft-105.17ft)',
    '16',
    '12-#6',
    57,
    'ROOF LEVEL',
    '(90.83ft-90.91ft)',
    '16',
    '12-#6',
    111,
    'ROOF LEVEL',
    '(90.64ft-76.40ft)',
    '16',
    '12-#6',
    112,
    'ROOF LEVEL',
    '(91.44ft-46.16ft)',
    '16',
    '12-#6',
    91,
    'ROOF LEVEL',
    '(91.44ft-32.83ft)',
    '16',
    '12-#6',
    118,
    'ROOF LEVEL',
    '(70.27ft-105.17ft)',
    '16',
    '12-#6',
    61,
    'ROOF LEVEL',
    '(70.24ft-90.91ft)',
    '16',
    '12-#6',
    164,
    'ROOF LEVEL',
    '(70.26ft-76.05ft)',
    '16',
    '12-#6',
    117,
    'ROOF LEVEL',
    '(71.24ft-57.01ft)',
    '16',
    '12-#6',
    259,
    'ROOF LEVEL',
    '(73.18ft-32.83ft)',
    '16',
    '12-#6',
    190,
    'ROOF LEVEL',
    '(45.44ft-105.17ft)',
    '16',
    '12-#6',
    61,
    'ROOF LEVEL',
    '(45.47ft-90.94ft)',
    '16',
    '12-#6',
    164,
    'ROOF LEVEL',
    '(45.45ft-76.54ft)',
    '16',
    '12-#6',
    428,
    'ROOF LEVEL',
    '(45.45ft-55.06ft)',
    '16',
    '12-#6',
    566,
    'ROOF LEVEL',
    '(45.45ft-32.83ft)',
    '16',
    '12-#6',
    283,
    'ROOF LEVEL',
    '(30.85ft-18.85ft)',
    '16',
    '12-#6',
    161,
    'ROOF LEVEL',
    '(30.85ft--1.14ft)',
    '16',
    '12-#6',
    109,
    'ROOF LEVEL',
    '(24.41ft-105.17ft)',
    '16',
    '12-#6',
    61,
    'ROOF LEVEL',
    '(24.41ft-90.91ft)',
    '16',
    '12-#6',
    177,
    'ROOF LEVEL',
    '(15.81ft-18.85ft)',
    '16',
    '12-#6',
    173,
    'ROOF LEVEL',
    '(15.34ft--1.03ft)',
    '16',
    '12-#6',
    49,
  

In [11]:
schedule = dict()
i = 0
while i < len(raw_data):
    key = f'{raw_data[i]}'
    
    if schedule.get(key) is None:
        schedule[key] = dict()

        schedule[key][raw_data[i+1]] = []
        schedule[key][raw_data[i+1]].append(raw_data[i+2])
        schedule[key][raw_data[i+1]].append(raw_data[i+3])
        schedule[key][raw_data[i+1]].append(raw_data[i+4])
        

    else:
        schedule[key][raw_data[i+1]] = []
        schedule[key][raw_data[i+1]].append(raw_data[i+2])
        schedule[key][raw_data[i+1]].append(raw_data[i+3])
        schedule[key][raw_data[i+1]].append(raw_data[i+4])
    
    i += 5
print(schedule)

{
    'BULKHEAD ROOF': {'(45.45ft-76.54ft)': ['16', '12-#6', 68], '(45.45ft-55.06ft)': ['16', '12-#6', 97]},
    'BULKHEAD': {
        '(24.41ft-90.91ft)': ['16', '12-#6', 96],
        '(110.64ft-90.91ft)': ['16', '12-#6', 51],
        '(110.83ft-76.45ft)': ['16', '12-#6', 34],
        '(111.09ft-46.16ft)': ['16', '12-#6', 67],
        '(110.90ft-32.83ft)': ['16', '12-#6', 45],
        '(90.83ft-90.91ft)': ['16', '12-#6', 76],
        '(90.64ft-76.40ft)': ['16', '12-#6', 60],
        '(91.44ft-46.16ft)': ['16', '12-#6', 78],
        '(91.44ft-32.83ft)': ['16', '12-#6', 59],
        '(70.24ft-90.91ft)': ['16', '12-#6', 67],
        '(70.26ft-76.05ft)': ['16', '12-#6', 84],
        '(71.24ft-57.01ft)': ['16', '12-#6', 132],
        '(73.18ft-32.83ft)': ['16', '12-#6', 129],
        '(45.47ft-90.94ft)': ['16', '12-#6', 67],
        '(45.45ft-76.54ft)': ['16', '12-#6', 112],
        '(45.45ft-55.06ft)': ['16', '12-#6', 192],
        '(45.45ft-32.83ft)': ['16', '12-#6', 140],
        '(30.85ft-18.85ft)': ['16', '12-#6', 80],
        '(30.85ft--1.14ft)': ['16', '12-#6', 53],
        '(15.81ft-18.85ft)': ['16', '12-#6', 90],
        '(15.34ft--1.03ft)': ['16', '12-#6', 31],
        '(-0.50ft-18.84ft)': ['16', '12-#6', 65],
        '(-0.58ft--0.58ft)': ['16', '12-#6', 60]
    },
    'ROOF LEVEL': {
        '(123.03ft-104.83ft)': ['16', '12-#6', 26],
        '(121.81ft-46.87ft)': ['16', '12-#6', 21],
        '(121.51ft-32.71ft)': ['16', '12-#6', 26],
        '(110.46ft-105.17ft)': ['16', '12-#6', 45],
        '(110.64ft-90.91ft)': ['16', '12-#6', 126],
        '(110.83ft-76.45ft)': ['16', '12-#6', 77],
        '(111.09ft-46.16ft)': ['16', '12-#6', 137],
        '(110.90ft-32.83ft)': ['16', '12-#6', 97],
        '(91.01ft-105.17ft)': ['16', '12-#6', 57],
        '(90.83ft-90.91ft)': ['16', '12-#6', 111],
        '(90.64ft-76.40ft)': ['16', '12-#6', 112],
        '(91.44ft-46.16ft)': ['16', '12-#6', 91],
        '(91.44ft-32.83ft)': ['16', '12-#6', 118],
        '(70.27ft-105.17ft)': ['16', '12-#6', 61],
        '(70.24ft-90.91ft)': ['16', '12-#6', 164],
        '(70.26ft-76.05ft)': ['16', '12-#6', 117],
        '(71.24ft-57.01ft)': ['16', '12-#6', 259],
        '(73.18ft-32.83ft)': ['16', '12-#6', 190],
        '(45.44ft-105.17ft)': ['16', '12-#6', 61],
        '(45.47ft-90.94ft)': ['16', '12-#6', 164],
        '(45.45ft-76.54ft)': ['16', '12-#6', 428],
        '(45.45ft-55.06ft)': ['16', '12-#6', 566],
        '(45.45ft-32.83ft)': ['16', '12-#6', 283],
        '(30.85ft-18.85ft)': ['16', '12-#6', 161],
        '(30.85ft--1.14ft)': ['16', '12-#6', 109],
        '(24.41ft-105.17ft)': ['16', '12-#6', 61],
        '(24.41ft-90.91ft)': ['16', '12-#6', 177],
        '(15.81ft-18.85ft)': ['16', '12-#6', 173],
        '(15.34ft--1.03ft)': ['16', '12-#6', 49],
        '(1.31ft-104.84ft)': ['16', '12-#6', 42],
        '(1.02ft-90.91ft)': ['16', '12-#6', 70],
        '(-0.50ft-18.84ft)': ['16', '12-#6', 161],
        '(-0.58ft--0.58ft)': ['16', '12-#6', 88]
    },
    'LEVEL 33': {
        '(123.03ft-104.83ft)': ['16', '12-#6', 50],
        '(121.81ft-46.87ft)': ['16', '12-#6', 30],
        '(121.51ft-32.71ft)': ['16', '12-#6', 51],
        '(110.46ft-105.17ft)': ['16', '12-#6', 84],
        '(110.64ft-90.91ft)': ['16', '12-#6', 170],
        '(110.83ft-76.45ft)': ['16', '12-#6', 93],
        '(111.09ft-46.16ft)': ['16', '12-#6', 154],
        '(110.90ft-32.83ft)': ['16', '12-#6', 90],
        '(91.01ft-105.17ft)': ['16', '12-#6', 106],
        '(90.83ft-90.91ft)': ['16', '12-#6', 126],
        '(90.64ft-76.40ft)': ['16', '12-#6', 121],
        '(91.44ft-46.16ft)': ['16', '12-#6', 125],
        '(91.44ft-32.83ft)': ['16', '12-#6', 144],
        '(70.27ft-105.17ft)': ['16', '12-#6', 100],
        '(70.24ft-90.91ft)': ['16', '12-#6', 142],
        '(70.26ft-76.05ft)': ['16', '12-#6', 383],
        '(71.24ft-57.01ft)': ['16', '12-#6', 507],
        '(73.18ft-32.83ft)': ['16', '12-#6', 258],
        '(45.44ft-105.17ft)': ['16', '12-#6', 113],
       

In [16]:
reformed_dict = {}
for outerKey, innerDict in schedule.items():
    for innerKey, values in innerDict.items():
        reformed_dict[(outerKey, innerKey)] = values
  

multiIndex_df = pd.DataFrame(reformed_dict)
multiIndex_df.rename(index={0: 'Size', 1: 'Rebar', 2: 'Load'}, inplace=True)

In [17]:
multiIndex_df

BULKHEAD ROOF                            BULKHEAD  \
      (45.45ft-76.54ft) (45.45ft-55.06ft) (24.41ft-90.91ft)   
Size                 16                16                16   
Rebar             12-#6             12-#6             12-#6   
Load                 68                97                96   

                                                                \
      (110.64ft-90.91ft) (110.83ft-76.45ft) (111.09ft-46.16ft)   
Size                  16                 16                 16   
Rebar              12-#6              12-#6              12-#6   
Load                  51                 34                 67   

                                                              \
      (110.90ft-32.83ft) (90.83ft-90.91ft) (90.64ft-76.40ft)   
Size                  16                16                16   
Rebar              12-#6             12-#6             12-#6   
Load                  45                76                60   

                         ...         1ST FLOOR                    \
      (91.44ft-46.16ft)  ... (30.85ft-18.85ft) (30.85ft--1.14ft)   
Size                 16  ...                20                16   
Rebar             12-#6  ...            12-#10             12-#6   
Load                 78  ...              2314              1377   

                                                              \
      (24.41ft-100.17ft) (24.41ft-90.91ft) (15.81ft-18.85ft)   
Size                  24                24                22   
Rebar             18-#11            18-#11            18-#10   
Load                3062              2368              2816   

                                                                              \
      (15.34ft--1.03ft) (1.21ft-100.17ft) (1.02ft-90.91ft) (-0.50ft-18.84ft)   
Size                 16                20               20                16   
Rebar             12-#8             12-#6            12-#7             12-#9   
Load               1775              2022             1385              1936   

                         
      (-0.58ft--0.58ft)  
Size                 16  
Rebar             12-#8  
Load               1775  

[3 rows x 1211 columns]

In [18]:
multiIndex_df.loc[:, 'BULKHEAD']

,(24.41ft-90.91ft),(110.64ft-90.91ft),(110.83ft-76.45ft),(111.09ft-46.16ft),(110.90ft-32.83ft),(90.83ft-90.91ft),(90.64ft-76.40ft),(91.44ft-46.16ft),(91.44ft-32.83ft),(70.24ft-90.91ft),...,(45.47ft-90.94ft),(45.45ft-76.54ft),(45.45ft-55.06ft),(45.45ft-32.83ft),(30.85ft-18.85ft),(30.85ft--1.14ft),(15.81ft-18.85ft),(15.34ft--1.03ft),(-0.50ft-18.84ft),(-0.58ft--0.58ft)
Size,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16
Rebar,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,...,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6
Load,96,51,34,67,45,76,60,78,59,67,...,67,112,192,140,80,53,90,31,65,60


In [20]:
schedule.keys()

dict_keys(['BULKHEAD ROOF', 'BULKHEAD', 'ROOF LEVEL', 'LEVEL 33', 'LEVEL 32', 'LEVEL 31', 'LEVEL 30', 'LEVEL 29', 'LEVEL 28', 'LEVEL 27', 'LEVEL 26', 'LEVEL 25', 'LEVEL 24', 'LEVEL 23', 'LEVEL 22', 'LEVEL 21', 'LEVEL 20', 'LEVEL 19', 'LEVEL 18', 'LEVEL 17', 'LEVEL 16', 'LEVEL 15', 'LEVEL 14', 'LEVEL 13', 'LEVEL 12', 'LEVEL 11', 'LEVEL 10', 'LEVEL 9', 'LEVEL 8', 'LEVEL 7', 'LEVEL 6', 'LEVEL 5', 'LEVEL 4', 'LEVEL 3', 'LEVEL 2', '1ST MEZZANINE', '1ST FLOOR'])

In [22]:
final = []
for story in schedule.keys():
    story_df = multiIndex_df.loc[:, story]
    story_df['story'] = story
    final.append(story_df)
schedule_df = pd.concat(final)

/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_37425/2012737168.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_37425/2012737168.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  story_df['story'] = story
/var/folders/g8/yczkcpmd20xfx96sfnfxmnpm0000gn/T/ipykernel_37425/2012737168.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [23]:
schedule_df

,(45.45ft-76.54ft),(45.45ft-55.06ft),story,(24.41ft-90.91ft),(110.64ft-90.91ft),(110.83ft-76.45ft),(111.09ft-46.16ft),(110.90ft-32.83ft),(90.83ft-90.91ft),(90.64ft-76.40ft),...,(121.22ft-18.73ft),(120.24ft--1.72ft),(110.53ft-18.85ft),(110.52ft--1.72ft),(91.44ft-18.85ft),(91.44ft--1.72ft),(73.97ft-18.85ft),(74.61ft--1.72ft),(45.45ft-18.85ft),(46.16ft--1.23ft)
Size,16,16,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rebar,12-#6,12-#6,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Load,68,97,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Size,16,16,BULKHEAD,16,16,16,16,16,16,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rebar,12-#6,12-#6,BULKHEAD,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Rebar,16-#11,22-#11,1ST MEZZANINE,18-#11,NaN,NaN,12-#7,12-#7,NaN,NaN,...,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6
Load,3732,4994,1ST MEZZANINE,2262,NaN,NaN,1767,1505,NaN,NaN,...,52,38,119,76,130,84,164,100,238,85
Size,"32""","36""",1ST FLOOR,24,NaN,NaN,16,"28""",NaN,NaN,...,16,16,"24""",16,"24""",16,"24""",16,"24""",16
Rebar,16-#11,22-#11,1ST FLOOR,18-#11,NaN,NaN,12-#7,12-#7,NaN,NaN,...,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6,12-#6


In [24]:
schedule_df = schedule_df.reindex(schedule_df.columns.sort_values(), axis=1)
story = schedule_df.story
schedule_df = schedule_df.drop(columns=['story'])
schedule_df.insert(loc=0, column='story', value=story)
schedule_df

,story,(-0.50ft-18.84ft),(-0.58ft--0.58ft),(1.02ft-90.91ft),(1.21ft-100.17ft),(1.31ft-104.84ft),(110.46ft-105.17ft),(110.52ft--1.72ft),(110.53ft-100.20ft),(110.53ft-18.85ft),...,(74.61ft--1.72ft),(90.61ft-73.87ft),(90.64ft-76.40ft),(90.83ft-90.91ft),(90.94ft-100.20ft),(91.01ft-105.17ft),(91.44ft--1.72ft),(91.44ft-18.85ft),(91.44ft-32.83ft),(91.44ft-46.16ft)
Size,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rebar,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Load,BULKHEAD ROOF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Size,BULKHEAD,16,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,16,16,NaN,NaN,NaN,NaN,16,16
Rebar,BULKHEAD,12-#6,12-#6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12-#6,12-#6,NaN,NaN,NaN,NaN,12-#6,12-#6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Rebar,1ST MEZZANINE,12-#9,12-#8,12-#7,12-#6,NaN,NaN,12-#6,16-#11,12-#6,...,12-#6,12-#7,NaN,NaN,24-#11,NaN,12-#6,12-#6,12-#7,18-#11
Load,1ST MEZZANINE,1873,1717,1326,1966,NaN,NaN,76,3229,119,...,100,2258,NaN,NaN,3975,NaN,84,130,1697,2694
Size,1ST FLOOR,16,16,20,20,NaN,NaN,16,26,"24""",...,16,24,NaN,NaN,28,NaN,16,"24""","28""",20
Rebar,1ST FLOOR,12-#9,12-#8,12-#7,12-#6,NaN,NaN,12-#6,16-#11,12-#6,...,12-#6,12-#7,NaN,NaN,24-#11,NaN,12-#6,12-#6,12-#7,18-#11


In [23]:
schedule_df.to_excel('17605_240328_dk_col_schedule.xlsx')

In [17]:
wb = openpyxl.Workbook()
sheet = wb.active
stories = column_db_column_data.Story_Label.unique()
xy_coords = column_db_column_data.Grid_Label.unique()

# write stories
a = 1
for i in range(len(stories)):
    cellA = f'A{a+1}'
    sheet[cellA] = stories[len(stories) - 1 - i]   
    a += 3

# write column B
params = ['LOAD', 'SIZE', 'BAR']

for j in range(len(stories)):
    for i, val in enumerate(params):
        cellB = f'B{i+2+j*3}'
        sheet[cellB] = val

# write row 1 <PICK UP FROM HERE>
# for i in range(len(xy_coords)):
#     cell1 = f'A
wb.save('column_schedule.xlsx')

In [18]:
column_db_column_data.Grid_Label.unique()

array(['82.71ft - 126.34ft to 82.46ft - 126.34ft', '82.46ft - 84.59ft',
       '82.71ft - 133.65ft', '93.21ft - 76.30ft',
       '98.41ft - 115.36ft to 98.41ft - 116.80ft',
       '98.74ft - 134.15ft to 98.74ft - 133.65ft',
       '106.01ft - 72.93ft to 105.49ft - 72.65ft', '111.04ft - 133.65ft',
       '111.33ft - 115.36ft to 111.33ft - 115.80ft',
       '111.72ft - 91.91ft to 111.33ft - 92.11ft',
       '119.09ft - 99.18ft to 119.33ft - 99.70ft', '125.26ft - 66.70ft',
       '126.21ft - 80.07ft', '126.99ft - 133.65ft to 126.57ft - 134.16ft',
       '126.90ft - 115.36ft to 126.90ft - 115.80ft',
       '137.89ft - 134.16ft',
       '139.63ft - 115.36ft to 139.63ft - 115.80ft',
       '142.28ft - 92.21ft to 142.64ft - 92.74ft', '146.10ft - 60.87ft',
       '151.90ft - 80.18ft to 152.38ft - 80.07ft', '153.63ft - 134.16ft',
       '165.24ft - 98.46ft to 162.57ft - 97.96ft', '162.81ft - 55.41ft',
       '162.91ft - 115.36ft to 162.91ft - 115.91ft',
       '167.30ft - 134.16ft', '173.13ft -